In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from torch.utils.data import random_split
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [4]:
data_folder = 'C:/Users/A/Desktop/Training_data/customData'
parameters_file = data_folder + '/0_Data.txt'

#calculate for normalize(mean,std)
def load_and_combine_data(data_folder):
    file_paths = [os.path.join(data_folder, f) for f in os.listdir(data_folder) if f.endswith('9.txt')]
    all_data = []

    for file_path in file_paths:
        #Loading arrary data from each txt file
        data = np.loadtxt(file_path)
        all_data.append(data)

    #combine all data to one file
    combined_data = np.concatenate(all_data, axis=0)
    return combined_data

def calculate_statistics(data):
    mean = np.mean(data)
    std = np.std(data)
    return mean, std

combined_data = load_and_combine_data(data_folder)
mean, std = calculate_statistics(combined_data)

In [5]:
#check the mean and std 
print(mean,std)

-0.029182019126922178 0.11373121119680919


In [6]:
class CustomDataset(Dataset):
    def __init__(self, data_folder, parameters_file, mean, std):
        self.data_folder = data_folder
        self.parameters = self.read_parameters(parameters_file)
        self.transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
            transforms.Normalize(mean, std)
        ])
        self.data_numbers = [data_num for data_num in self.parameters.keys() if data_num.endswith('9')]

#Read '0_Data.txt'
    def read_parameters(self, file_path):
        parameters = {}
        with open(file_path, 'r') as file:
            for index, line in enumerate(file):
                if index == 0:  # Skip 1st row
                    continue
                parts = line.strip().split(',')
                data_number = parts[0]
                params = np.array(parts[1:4], dtype=np.float32)
                parameters[data_number] = params
        return parameters

    def __len__(self):
        return len(self.data_numbers)

    def __getitem__(self, idx):
        data_number = self.data_numbers[idx]
        matrix_path = os.path.join(self.data_folder, f'{data_number}.txt')
        matrix = np.loadtxt(matrix_path)  # Load matrix from a txt file
        matrix = torch.from_numpy(matrix).float().unsqueeze(0)  # 2D -> 3D tensor
        matrix = self.transform(matrix)
        params = self.parameters[data_number]
        return matrix, params

custom_dataset = CustomDataset(data_folder=data_folder, parameters_file=parameters_file, mean=mean, std=std)

In [9]:
#check the custom_dataset
print(custom_dataset[0])

(tensor([[[-3.5984, -3.5984, -3.5983,  ...,  0.2767,  0.2770,  0.2772],
         [-3.5985, -3.5983, -3.5980,  ...,  0.2767,  0.2770,  0.2772],
         [-3.5987, -3.5982, -3.5975,  ...,  0.2767,  0.2770,  0.2772],
         ...,
         [ 0.2767,  0.2767,  0.2768,  ...,  0.2785,  0.2784,  0.2784],
         [ 0.2770,  0.2770,  0.2771,  ...,  0.2784,  0.2784,  0.2783],
         [ 0.2772,  0.2772,  0.2772,  ...,  0.2784,  0.2783,  0.2782]],

        [[-3.5984, -3.5984, -3.5983,  ...,  0.2767,  0.2770,  0.2772],
         [-3.5985, -3.5983, -3.5980,  ...,  0.2767,  0.2770,  0.2772],
         [-3.5987, -3.5982, -3.5975,  ...,  0.2767,  0.2770,  0.2772],
         ...,
         [ 0.2767,  0.2767,  0.2768,  ...,  0.2785,  0.2784,  0.2784],
         [ 0.2770,  0.2770,  0.2771,  ...,  0.2784,  0.2784,  0.2783],
         [ 0.2772,  0.2772,  0.2772,  ...,  0.2784,  0.2783,  0.2782]],

        [[-3.5984, -3.5984, -3.5983,  ...,  0.2767,  0.2770,  0.2772],
         [-3.5985, -3.5983, -3.5980,  ...,  

d:\anaconda3\envs\custom\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [20]:
data_loader = DataLoader(custom_dataset, batch_size=32, shuffle=True)

In [10]:
'''
import matplotlib.pyplot as plt

# 첫 번째 샘플 로드
first_sample = custom_dataset[0]
image, params = first_sample

# 이미지가 CUDA 텐서인 경우, CPU로 이동
if image.is_cuda:
    image = image.cpu()

# 이미지를 NumPy 배열로 변환하고 차원을 재배열
image_numpy = image.numpy().transpose(1, 2, 0)

# 이미지 시각화
plt.imshow(image_numpy)
plt.title(f'Parameters: {params}')
plt.show()
'''

"\nimport matplotlib.pyplot as plt\n\n# 첫 번째 샘플 로드\nfirst_sample = custom_dataset[0]\nimage, params = first_sample\n\n# 이미지가 CUDA 텐서인 경우, CPU로 이동\nif image.is_cuda:\n    image = image.cpu()\n\n# 이미지를 NumPy 배열로 변환하고 차원을 재배열\nimage_numpy = image.numpy().transpose(1, 2, 0)\n\n# 이미지 시각화\nplt.imshow(image_numpy)\nplt.title(f'Parameters: {params}')\nplt.show()\n"

In [11]:
#train_dataset = CustomDataset(data_folder, parameters_file, mean, std, train=True)
#test_dataset = CustomDataset(data_folder, parameters_file, mean, std, train=False)

In [12]:
label_size = 3
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, label_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

d:\anaconda3\envs\custom\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\anaconda3\envs\custom\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
from torchsummary import summary
import torchvision.models as models

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [21]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(custom_dataset)}')
print('Finished Training')

d:\anaconda3\envs\custom\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Loss: 4132.230646306818
Epoch 2, Loss: 3026.1056389086175
Epoch 3, Loss: 2084.9740397135415
Epoch 4, Loss: 1336.360751065341
Epoch 5, Loss: 845.5816095525569
Epoch 6, Loss: 573.7845495975379
Epoch 7, Loss: 383.6577518347538
Epoch 8, Loss: 269.4897687507398
Epoch 9, Loss: 202.62509687019116
Epoch 10, Loss: 189.56262854373816
Finished Training


In [22]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(custom_dataset)}')
print('Finished Training')

Epoch 1, Loss: 123.03258005777995
Epoch 2, Loss: 150.73558460582387
Epoch 3, Loss: 109.36716969807942
Epoch 4, Loss: 148.57869119355172
Epoch 5, Loss: 152.52163696289062
Epoch 6, Loss: 142.3885407881303
Epoch 7, Loss: 128.35511826023912
Epoch 8, Loss: 150.210185657848
Epoch 9, Loss: 163.4852496060458
Epoch 10, Loss: 136.4463094075521
Epoch 11, Loss: 149.9955044370709
Epoch 12, Loss: 119.44866411613695
Epoch 13, Loss: 112.6716352520567
Epoch 14, Loss: 108.81981797651811
Epoch 15, Loss: 135.512858766498
Epoch 16, Loss: 103.10052490234375
Epoch 17, Loss: 164.6572889848189
Epoch 18, Loss: 110.62416770241477
Epoch 19, Loss: 119.20378043434836
Epoch 20, Loss: 117.67996447014086
Epoch 21, Loss: 113.7900698112719
Epoch 22, Loss: 128.53056844075522
Epoch 23, Loss: 125.4719887935754
Epoch 24, Loss: 115.56079170920633
Epoch 25, Loss: 109.93647534919508
Epoch 26, Loss: 119.23985313646722
Epoch 27, Loss: 103.03176394375888
Epoch 28, Loss: 104.22825761274858
Epoch 29, Loss: 98.83236046993372
Epoch 3